## Prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)

In [2]:
mesa = pd.read_csv('../mesa/data_processed/Y_BaselineX_processed_full.csv')

mesa = mesa.rename(columns = {'cvda': 'Y_tot', 
                             'F1_PC2': 'nSES',
                            'S1FAV': 'nFavFood',
                            'S1PAI': 'nPhysFac', 
                            'G_bla_rk': 'nRS', 
                            'chdiet': 'nutrition', 
                            'chphysact': 'PhysAct',
                            'income': 'FamIncome'})

diet_phys_map = {0:1, 1:2, 2:3}
mesa['nutrition'] = mesa['nutrition'].replace(diet_phys_map)
mesa['PhysAct'] = mesa['PhysAct'].replace(diet_phys_map)

mesa_bla = mesa[mesa['race'] == 3]

In [3]:
jhs = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')

jhs = jhs.rename(columns = {'event': 'Y_tot', 
                             'nbSESpc2score': 'nSES',
                            'S1FAV': 'nFavFood',
                            'S1PAI': 'nPhysFac', 
                            'G_bla_rk': 'nRS', 
                            'nutrition3cat': 'nutrition', 
                            'PA3cat': 'PhysAct',
                            'fmlyinc': 'FamIncome'})
jhs['nutrition'] = jhs['nutrition'].replace(diet_phys_map)
jhs['PhysAct'] = jhs['PhysAct'].replace(diet_phys_map)

## Mesa

In [7]:
y = mesa['Y_tot']

X = mesa[['nSES','nFavFood','nPhysFac', 'nRS',
          'FamIncome','nutrition', 'PhysAct',
          'age','gender','race','cig','cural','diabet','hdl','chol','sbp']]

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()
X4 = X.copy()
X5 = X.copy()
X6 = X.copy()

X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
X1 = pd.get_dummies(X1, columns=['nSES','nFavFood','nPhysFac', 'nRS',
                                 'FamIncome','nutrition', 'PhysAct',
                                 'gender','race','cig','cural','diabet','nSES_FamIncome'], 
                    drop_first=True)

X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']

In [8]:
logistic_model_with_moderation = sm.Logit(y, sm.add_constant(X1)).fit()
print(logistic_model_with_moderation.summary())

Optimization terminated successfully.
         Current function value: 0.409347
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Y_tot   No. Observations:                 6787
Model:                          Logit   Df Residuals:                     6749
Method:                           MLE   Df Model:                           37
Date:                Sat, 04 Nov 2023   Pseudo R-squ.:                 0.09941
Time:                        17:16:53   Log-Likelihood:                -2778.2
converged:                       True   LL-Null:                       -3084.9
Covariance Type:            nonrobust   LLR p-value:                1.523e-105
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -5.2108      0.409    -12.744      0.000      -6.012      -4.409
ag

In [ ]:
## issues:
## 2*3 = 3*2; inconsistent significance across dif subgroups